reference : https://www.kaggle.com/code/lusfernandotorres/text-summarization-with-large-language-models/notebook

In [ ]:
# !pip install datasets # Installing the datasets library (https://huggingface.co/docs/datasets/index)
# !pip install plotly
# pip install scipy
# pip install statsmodels
# !pip install evaluate # Installing the evaluate library (https://huggingface.co/docs/evaluate/main/en/index)
# pip install textblob
# pip install scikit-learn

# Importing Library

In [78]:
# Data Handling
import pandas as pd
import numpy as np
from datasets import Dataset, load_metric
import shutil

# Data Visualization
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
from IPython.display import display
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Statistics & Mathematics
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import shapiro, skew, anderson, kstest, gaussian_kde,spearmanr
import math

# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

# Transformers
from transformers import BartTokenizer, BartForConditionalGeneration      # BERT Tokenizer and architecture
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments         # These will help us to fine-tune our model
from transformers import pipeline                                         # Pipeline
from transformers import DataCollatorForSeq2Seq                           # DataCollator to batch the data 
import torch                                                              # PyTorch
import evaluate                                                           # Hugging Face's library for model evaluation


# Other NLP libraries
from textblob import TextBlob                                             # This is going to help us fix spelling mistakes in texts
from sklearn.feature_extraction.text import TfidfVectorizer               # This is going to helps identify the most common terms in the corpus
import re                                                                 # This library allows us to clean text data
import nltk                                                               # Natural Language Toolkit
nltk.download('punkt')                                                    # This divides a text into a list of sentences

In [80]:
# Configuring notebook
seed = 42
#paper_color =
#bg_color =
colormap = 'cividis'
template = 'plotly_dark'

In [81]:
# Checking if GPU is available
if torch.cuda.is_available():
    print("GPU is available. \nUsing GPU")
    device = torch.device('cuda')
else:
    print("GPU is not available. \nUsing CPU")
    device = torch.device('cpu')

GPU is available. 
Using GPU


In [82]:
def display_feature_list(features, feature_type):

    '''
    This function displays the features within each list for each type of data
    '''

    print(f"\n{feature_type} Features: ")
    print(', '.join(features) if features else 'None')

def describe_df(df):
    """
    This function prints some basic info on the dataset and
    sets global variables for feature lists.
    """

    global categorical_features, continuous_features, binary_features
    categorical_features = [col for col in df.columns if df[col].dtype == 'object']
    binary_features = [col for col in df.columns if df[col].nunique() <= 2 and df[col].dtype != 'object']
    continuous_features = [col for col in df.columns if df[col].dtype != 'object' and col not in binary_features]

    print(f"\n{type(df).__name__} shape: {df.shape}")
    print(f"\n{df.shape[0]:,.0f} samples")
    print(f"\n{df.shape[1]:,.0f} attributes")
    print(f'\nMissing Data: \n{df.isnull().sum()}')
    print(f'\nDuplicates: {df.duplicated().sum()}')
    print(f'\nData Types: \n{df.dtypes}')

    #negative_valued_features = [col for col in df.columns if (df[col] < 0).any()]
    #print(f'\nFeatures with Negative Values: {", ".join(negative_valued_features) if negative_valued_features else "None"}')

    display_feature_list(categorical_features, 'Categorical')
    display_feature_list(continuous_features, 'Continuous')
    display_feature_list(binary_features, 'Binary')

    print(f'\n{type(df).__name__} Head: \n')
    display(df.head(5))
    print(f'\n{type(df).__name__} Tail: \n')
    display(df.tail(5))

In [83]:
def histogram_boxplot(df,hist_color, box_color, height, width, legend, name):
    '''
    This function plots a Histogram and a Box Plot side by side

    Parameters:
    hist_color = The color of the histogram
    box_color = The color of the boxplots
    heigh and width = Image size
    legend = Either to display legend or not
    '''

    features = df.select_dtypes(include = [np.number]).columns.tolist()

    for feat in features:
        try:
            fig = make_subplots(
                rows=1,
                cols=2,
                subplot_titles=["Box Plot", "Histogram"],
                horizontal_spacing=0.2
            )

            density = gaussian_kde(df[feat])
            x_vals = np.linspace(min(df[feat]), max(df[feat]), 200)
            density_vals = density(x_vals)

            fig.add_trace(go.Scatter(x=x_vals, y = density_vals, mode = 'lines',
                                     fill = 'tozeroy', name="Density", line_color=hist_color), row=1, col=2)
            fig.add_trace(go.Box(y=df[feat], name="Box Plot", boxmean=True, line_color=box_color), row=1, col=1)

            fig.update_layout(title={'text': f'<b>{name} Word Count<br><sup><i>&nbsp;&nbsp;&nbsp;&nbsp;{feat}</i></sup></b>',
                                     'x': .025, 'xanchor': 'left'},
                             margin=dict(t=100),
                             showlegend=legend,
                             template = template,
                             #plot_bgcolor=bg_color,paper_bgcolor=paper_color,
                             height=height, width=width
                            )

            fig.update_yaxes(title_text=f"<b>Words</b>", row=1, col=1, showgrid=False)
            fig.update_xaxes(title_text="", row=1, col=1, showgrid=False)

            fig.update_yaxes(title_text="<b>Frequency</b>", row=1, col=2,showgrid=False)
            fig.update_xaxes(title_text=f"<b>Words</b>", row=1, col=2, showgrid=False)

            fig.show()
            print('\n')
        except Exception as e:
            print(f"An error occurred: {e}")

In [84]:
def plot_correlation(df, title, subtitle, height, width, font_size):
    '''
    This function is resposible to plot a correlation map among features in the dataset.

    Parameters:
    height = Define height
    width = Define width
    font_size = Define the font size for the annotations
    '''
    corr = np.round(df.corr(numeric_only = True), 2)
    mask = np.triu(np.ones_like(corr, dtype = bool))
    c_mask = np.where(~mask, corr, 100)

    c = []
    for i in c_mask.tolist()[1:]:
        c.append([x for x in i if x != 100])



    fig = ff.create_annotated_heatmap(z=c[::-1],
                                      x=corr.index.tolist()[:-1],
                                      y=corr.columns.tolist()[1:][::-1],
                                      colorscale = colormap)

    fig.update_layout(title = {'text': f"<b>{title} Heatmap<br><sup>&nbsp;&nbsp;&nbsp;&nbsp;<i>{subtitle}</i></sup></b>",
                                'x': .025, 'xanchor': 'left', 'y': .95},
                    margin = dict(t=210, l = 110),
                    yaxis = dict(autorange = 'reversed', showgrid = False),
                    xaxis = dict(showgrid = False),
                    template = template,
                    #plot_bgcolor=bg_color,paper_bgcolor=paper_color,
                    height = height, width = width)


    fig.add_trace(go.Heatmap(z = c[::-1],
                             colorscale = colormap,
                             showscale = True,
                             visible = False))
    fig.data[1].visible = True

    for i in range(len(fig.layout.annotations)):
        fig.layout.annotations[i].font.size = font_size

    fig.show()

In [85]:
def compute_tfidf(df_column, ngram_range=(1,1), max_features=15):
    vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english', ngram_range=ngram_range)
    x = vectorizer.fit_transform(df_column.fillna(''))
    df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
    return df_tfidfvect

In [86]:
pwd

'/data/ephemeral/home/code'

In [87]:
# Loading data
train = pd.read_csv('../data/use/train.csv')
test = pd.read_csv('../data/origin/test.csv')
val = pd.read_csv('../data/use/dev.csv')

In [88]:
# Extracting info on the training Dataframe
describe_df(train)


DataFrame shape: (12405, 4)

12,405 samples

4 attributes

Missing Data: 
fname       0
dialogue    0
summary     0
topic       0
dtype: int64

Duplicates: 0

Data Types: 
fname       object
dialogue    object
summary     object
topic       object
dtype: object

Categorical Features: 
fname, dialogue, summary, topic

Continuous Features: 
None

Binary Features: 
None

DataFrame Head: 



,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#: 안녕하세요, 피터스 박사님. 잘 지냈습니다, 감사합니다. 리키와 함께 백신 접종을 위해 왔습니다.\n#Person1#: 좋습니다. 백신 접종 기록을 보니, 리키는 이미 소아마비, 디프테리아, B형 간염 백신을 맞았군요. 그는 14개월이므로, 이제 A형 간염, 수두, 홍역 백신을 맞아야 합니다.\n#Person2#: 풍진과 볼거리는 어떻게 되나요?\n#Person1#: 지금은 이 백신들만 접종할 수 있고, 몇 주 후에 나머지를 접종할 수 있습니다.\n#Person2#: 좋습니다. 박사님, 저도 디프테리아 예방접종이 필요할 것 같아요. 마지막으로 맞은 게 아마도 15년 전이었던 것 같아요!\n#Person1#: 저희가 기록을 확인하고 간호사에게 부스터를 접종하도록 하겠습니다. 이제, 리키의 팔을 꽉 잡아주세요, 조금 찌릿할 수 있습니다.",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후 리키에게 백신을 접종했다.,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤 종류의 열쇠인가요?\n#Person1#: 5개의 열쇠와 작은 발 장식이 있어요.\n#Person2#: 안타깝네요! 저는 보지 못했습니다.\n#Person1#: 그럼, 찾는 데 도와주실 수 있나요? 제가 여기는 처음이라서요.\n#Person2#: 물론입니다. 기꺼이 도와드리겠습니다. 사라진 열쇠를 찾는 데 도와드리겠습니다.\n#Person1#: 정말 친절하시네요.\n#Person2#: 별 말씀을요. 어, 찾았어요.\n#Person1#: 오, 하느님 감사합니다! 어떻게 감사의 말씀을 드려야 할지 모르겠네요.\n#Person2#: 천만에요.","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#에게 도움을 청하고 있습니다.",열쇠 찾기
3,train_3,"#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person2#: 미안해, 너가 이미 알고 있다고 생각했어.\n#Person1#: 하지만 너가 그녀를 사랑한다는 걸 말해줘야 했어.\n#Person2#: 내가 하지 않았어?\n#Person1#: 너도 알고 있잖아, 말하지 않았다고.\n#Person2#: 그래, 지금 말해주고 있잖아.\n#Person1#: 그래, 하지만 너는 나에게 미리 말해줄 수 있었어.\n#Person2#: 네가 관심 있을 줄은 몰랐어.\n#Person1#: 진짜 농담이지. 어떻게 그녀와 결혼할 거라는 걸 말하지 않을 수 있어?\n#Person2#: 미안해, 나는 그게 중요하다고 생각하지 않았어.\n#Person1#: 아, 남자들이란! 다 똑같아.",#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실을 #Person1#에게 말하지 않았기 때문에 화가 났다.,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 나와 함께 해줄래?\n#Person2#: 그는 귀여워! 그는 타이거 우즈 같아 보여! 하지만, 나는 춤을 못 춰...\n#Person1#: 괜찮아. 내가 너에게 모든 올바른 동작을 보여줄게. 내 이름은 말릭이야.\n#Person2#: 만나서 반가워. 나는 웬이고, 이쪽은 니키야.\n#Person1#: 기분 어때, 비스타? 네 친구를 춤추는 곳으로 데려가도 괜찮을까?\n#Person2#: 네 발을 밟아도 상관없다면 그녀는 신경 쓰지 않을 거야.\n#Person1#: 좋아. 멋져! 가자!",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 동의한다.,댄스



DataFrame Tail: 



,fname,dialogue,summary,topic
12400,train_12455,"#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2#: 네, 저를 알아봐 주셔서 기쁩니다.\n#Person1#: 텔렉스에서 자신을 백발과 수염이 있는 남자라고 설명하셨잖아요? 그래서 어디에서든 쉽게 찾을 수 있었습니다. 제 소개부터 하겠습니다. 저는 황하 수입 수출 회사의 소스 매니저, 탄 링입니다. 만나서 반갑습니다.\n#Person2#: 만나서 반갑습니다.\n#Person1#: 그린 씨, 동행하지 않는 짐이 있나요?\n#Person2#: 아니요. 항상 이 가방이나 작은 여행 가방만 가지고 다닙니다.\n#Person1#: 그럼 가시죠. 이쪽이 정문으로 가는 길입니다. 짐을 들어드릴게요.\n#Person2#: 아니요, 여성이 제 짐을 들어주는 것은 상상도 못하겠습니다. 아직 그렇게 늙지 않았습니다.\n#Person1#: 물론이죠. 흰머리에도 불구하고 젊고 활기찬 모습이시네요, 그린 씨.\n#Person2#: 감사합니다. 그렇게 말씀해주셔서 기쁩니다.\n#Person1#: 차가 밖에서 우리를 호텔로 데려가기 위해 기다리고 있습니다. 저는 당신을 위해 더블린에서 가장 큰 피닉스 호텔에 스위트룸을 예약했습니다.\n#Person2#: 가장 큰 스위트룸을 말하는 건가요, 아니면 가장 큰 호텔을 말하는 건가요?\n#Person1#: 사실 호텔과 스위트룸 모두 꽤 큽니다. 하지만 가장 큰 것이 항상 최고는 아니죠.\n#Person2#: 맞는 말씀이네요.\n#Person1#: 그린 씨, 스위트룸을 보시면 알겠지만, 마음에 들지 않으면 더 좋은 것으로 바꾸거나 다른 호텔로 이동하실 수 있습니다.",탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다. 탄은 그린 씨를 위해 호텔에서 큰 스위트룸을 예약했습니다.,누군가를 태우다
12401,train_12456,"#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 했죠?\n#Person2#: 네, 그는 특히 우리가 늦지 않도록 요청했습니다. 우리 동요크 지사에서 몇몇 사람들이 오는데, 그들에게 좋은 인상을 남기고 싶어합니다. 어떻게 가실 건가요?\n#Person1#: 제 차를 타고 가려고 했는데, 고속도로에 공사가 있어서 지하철을 이용할 것 같습니다. 당신은요?\n#Person2#: 저도 지하철을 이용할 예정입니다. 같이 가는 건 어떨까요? 저는 컨퍼런스 센터에 한 번밖에 가본 적이 없어서 길을 잘 못 찾을 것 같아요.",#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼런스 센터로 지하철을 같이 타기로 계획했습니다.,컨퍼런스 센터
12402,train_12457,"#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶습니다.\n#Person1#: 찾아볼게요. 대형 차, 중형 차, 소형 차 중에서 선택하실 수 있습니다. 어떤 크기를 찾으시나요?\n#Person2#: 도시에서 혼자 여행할 예정이라 소형 차로 괜찮습니다. 하루에 얼마인가요?\n#Person1#: 소형 차는 하루에 $40입니다. 차를 얼마나 오래 빌리실 건가요?\n#Person2#: 5일 동안입니다.\n#Person1#: 알겠습니다. 운전 면허증과 신용카드를 볼 수 있을까요?\n#Person2#: 네, 여기 있습니다.\n#Person1#: 이 카드로 결제 하시겠습니까?\n#Person2#: 그렇게 해주세요.",#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12403,train_12458,"#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2#: 음, 어제 엄마가 일자리를 잃었어.\n#Person1#: 그런 일이 있었다니 유감스럽게 생각해. 이번 해에 도시 등록 실업률이 4%에 달하고 그 중 절반 이상이 여성이라고 들었어.\n#Person2#: 일자리 시장에서 공급이 수요를 초과하고 있고, 여성들이 일반적으로 불리한 위치에 있어.\n#Person1#: 그렇지, 맞아. 그럼 그녀는 어떻게 할 거야?\n#Person2#: 음. 그녀는 커뮤니티에서 가사 청소나 아이 돌보기 같은 일을 생각하고 있어.\n#Person1#: 나쁘지 않아, 단기적인 대안이 될 수 있을 거야. 시장 상황이 좋아지면 엄마는 다른 일자리를 찾을 수 있을 거야. 결국엔 잘 될 거야.\n#Person2#: 그녀가 우울해하지 않았으면 좋겠어.\n#Person1#: 인터넷에서 일자리 정보를 찾아보는 건 어때?\n#Person2#: 좋은 생각이야, 고마워.",#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 않기를 바란다. #Person1#은 #Person2#에게 인터넷에서 일자리 정보를 찾아보는 것을 제안한다.,실직
12404,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 타고 갈 거예요. 오늘 가방을 싸도 될까요?\n#Person2#: 네, 그게 좋을 것 같아요.\n#Person1#: 알겠어요. 어떤 옷을 가져가야 할까요? 거기 날씨가 더운 건 알고 있어요.\n#Person2#: 네, 하지만 비가 많이 와요. 비가 오면 우산이나 재킷을 빌릴 수 있어요. 티셔츠만 몇 장 싸가세요.\n#Person1#: 알겠어요. 그럼 공항에서 누가 저를 만나러 오나요?\n#Person2#: 이 삼촌과 왕 이모는 바쁠 테니까, 사촌 수잔이 당신을 데리러 갈 수 있을 거예요.",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 #Person2#에게 조언을 구합니다.,짐 싸기


In [89]:
df_text_lenght = pd.DataFrame() # Creating an empty dataframe
for feat in categorical_features: # Iterating through features --> Dialogue & Summary
    df_text_lenght[feat] = train[feat].apply(lambda x: len(str(x).split())) #  Counting words for each feature

# Plotting histogram-boxplot
histogram_boxplot(df_text_lenght,'#89c2e0', '#d500ff', 600, 1000, True, 'Train Dataset')

An error occurred: The data appears to lie in a lower-dimensional subspace of the space in which it is expressed. This has resulted in a singular data covariance matrix, which cannot be treated using the algorithms implemented in `gaussian_kde`. Consider performing principle component analysis / dimensionality reduction and using `gaussian_kde` with the transformed data.


In [90]:
train

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#: 안녕하세요, 피터스 박사님. 잘 지냈습니다, 감사합니다. 리키와 함께 백신 접종을 위해 왔습니다.\n#Person1#: 좋습니다. 백신 접종 기록을 보니, 리키는 이미 소아마비, 디프테리아, B형 간염 백신을 맞았군요. 그는 14개월이므로, 이제 A형 간염, 수두, 홍역 백신을 맞아야 합니다.\n#Person2#: 풍진과 볼거리는 어떻게 되나요?\n#Person1#: 지금은 이 백신들만 접종할 수 있고, 몇 주 후에 나머지를 접종할 수 있습니다.\n#Person2#: 좋습니다. 박사님, 저도 디프테리아 예방접종이 필요할 것 같아요. 마지막으로 맞은 게 아마도 15년 전이었던 것 같아요!\n#Person1#: 저희가 기록을 확인하고 간호사에게 부스터를 접종하도록 하겠습니다. 이제, 리키의 팔을 꽉 잡아주세요, 조금 찌릿할 수 있습니다.",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후 리키에게 백신을 접종했다.,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤 종류의 열쇠인가요?\n#Person1#: 5개의 열쇠와 작은 발 장식이 있어요.\n#Person2#: 안타깝네요! 저는 보지 못했습니다.\n#Person1#: 그럼, 찾는 데 도와주실 수 있나요? 제가 여기는 처음이라서요.\n#Person2#: 물론입니다. 기꺼이 도와드리겠습니다. 사라진 열쇠를 찾는 데 도와드리겠습니다.\n#Person1#: 정말 친절하시네요.\n#Person2#: 별 말씀을요. 어, 찾았어요.\n#Person1#: 오, 하느님 감사합니다! 어떻게 감사의 말씀을 드려야 할지 모르겠네요.\n#Person2#: 천만에요.","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#에게 도움을 청하고 있습니다.",열쇠 찾기
3,train_3,"#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person2#: 미안해, 너가 이미 알고 있다고 생각했어.\n#Person1#: 하지만 너가 그녀를 사랑한다는 걸 말해줘야 했어.\n#Person2#: 내가 하지 않았어?\n#Person1#: 너도 알고 있잖아, 말하지 않았다고.\n#Person2#: 그래, 지금 말해주고 있잖아.\n#Person1#: 그래, 하지만 너는 나에게 미리 말해줄 수 있었어.\n#Person2#: 네가 관심 있을 줄은 몰랐어.\n#Person1#: 진짜 농담이지. 어떻게 그녀와 결혼할 거라는 걸 말하지 않을 수 있어?\n#Person2#: 미안해, 나는 그게 중요하다고 생각하지 않았어.\n#Person1#: 아, 남자들이란! 다 똑같아.",#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실을 #Person1#에게 말하지 않았기 때문에 화가 났다.,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 나와 함께 해줄래?\n#Person2#: 그는 귀여워! 그는 타이거 우즈 같아 보여! 하지만, 나는 춤을 못 춰...\n#Person1#: 괜찮아. 내가 너에게 모든 올바른 동작을 보여줄게. 내 이름은 말릭이야.\n#Person2#: 만나서 반가워. 나는 웬이고, 이쪽은 니키야.\n#Person1#: 기분 어때, 비스타? 네 친구를 춤추는 곳으로 데려가도 괜찮을까?\n#Person2#: 네 발을 밟아도 상관없다면 그녀는 신경 쓰지 않을 거야.\n#Person1#: 좋아. 멋져! 가자!",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 동의한다.,댄스
...,...,...,...,...
12400,train_12455,"#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2#: 네, 저를 알아봐 주셔서 기쁩니다.\n#Person1#: 텔렉스에서 자신을 백발과 수염이 있는 남자라고 설명하셨잖아요? 그래서 어디에서든 쉽게 찾을 수 있었습니다. 제 소개부터 하겠습니다. 저는 황하 수입 수출 회사의 소스 매니저, 탄 링입니다. 만나서 반갑습니다.\n#Person2#: 만나서 반갑습니다.\n#Person1#: 그린 씨, 동행하지 않는 짐이 있나요?\n#Person2#: 아니요. 항상 이 가방이나 작은 여행 가방만 가지고 다닙니다.\n#Person1#: 그럼 가시죠. 이쪽이 정문으로 가는 길입니다. 짐을 들어드릴게요.\n#Person2#: 아니요, 여성이 제 짐을 들어주는 것은 상상도 못하겠습니다. 아직 그렇게 늙지 않았습니다.\n#Person1#: 물론이죠. 흰머리에도 불구하고 젊고 활기찬 모습이시네요, 그린 씨.\n#Person2#: 감사합니다. 그렇게 말씀해주셔서 기쁩니다.\n#Person1#: 차가 밖에서 우리를 호텔로 데려가기 위해 기다리고 있습니다. 저는 당신을 위해 더블린에서 가장 큰 피닉스 호텔에 스위트룸을 예약했습니다.\n#Person2#: 가장 큰 스위트룸을 말하는 건가요, 아니면 가장 큰 호텔을 말하는 건가요?\n#Person1#: 사실 호텔과 스위트룸 모두 꽤 큽니다. 하지만 가장 큰 것이 항상 최고는 아니죠.\n#Person2#: 맞는 말씀이네요.\n#Person1#: 그린 씨, 스위트룸을 보시면 알겠지만, 마음에 들지 않으면 더 좋은 것으로 바꾸거나 다른 호텔로 이동하실 수 있습니다.",탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다. 탄은 그린 씨를 위해 호텔에서 큰 스위트룸을 예약했습니다.,누군가를 태우다
12401,train_12456,"#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 했죠?\n#Person2#: 네, 그는 특히 우리가 늦지 않도록 요청했습니다. 우리 동요크 지사에서 몇몇 사람들이 오는데, 그들에게 좋은 인상을 남기고 싶어합니다. 어떻게 가실 건가요?\n#Person1#: 제 차를 타고 가려고 했는데, 고속도로에 공사가 있어서 지하철을 이용할 것 같습니다. 당신은요?\n#Person2#: 저도 지하철을 이용할 예정입니다. 같이 가는 건 어떨까요? 저는 컨퍼런스 센터에 한 번밖에 가본 적이 없어서 길을 잘 못 찾을 것 같아요.",#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼런스 센터로 지하철을 같이 타기로 계획했습니다.,컨퍼런스 센터
12402,train_12457,"#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶

평균적으로 대화는 약 94개의 단어로 구성됩니다. 대화당 300단어가 넘는 매우 광범위한 텍스트를 포함하는 이상치가 있습니다.

요약은 자연적으로 평균 약 20단어로 구성된 짧은 텍스트이지만 광범위한 요약이 포함된 일부 특이점도 있습니다.

또한 scikit-learn의 TfidfVectorizer를 사용하여 사용 가능한 대화 및 요약에 대한 추가 정보를 추출할 수도 있습니다. 이 함수는 우리가 max_features 매개변수를 사용하여 선택하는 코퍼스에서 상위 n개의 가장 빈번한 용어가 있는 데이터프레임을 제공합니다.

이 데이터프레임에서 각 열은 전체 코퍼스에서 n개의 가장 빈번한 용어를 나타내고, 각 행은 train과 같은 원래 데이터프레임의 항목 하나를 나타냅니다. 각 항목의 각 용어에 대해 연관된 TF-IDF 점수를 볼 수 있습니다. 이는 다른 모든 대화 또는 요약에서의 빈도와 관련하여 특정 대화 또는 요약에서 용어의 관련성을 정량화합니다.

또한 ngram_range 매개변수를 사용하여 가장 자주 사용되는 단어(unigrams), 가장 자주 사용되는 두 단어 시퀀스(bigrams) 및 가장 자주 사용되는 세 단어 시퀀스(trigrams)를 선택합니다. stop_words = 'english' 매개변수는 영어의 일반적인 불용어("and", "of" 등과 같이 전체 문맥에 크게 영향을 미치지 않는 단어)를 필터링하는 데 도움이 됩니다.

가장 자주 사용되는 용어를 측정한 후 이러한 용어 간의 상관 관계를 표시하는 히트맵을 작성하겠습니다. 이는 대화에서 두 단어가 얼마나 자주 함께 사용되는지 이해하는 데 도움이 될 수 있습니다. 예를 들어, "we"라는 단어가 있을 때 "will"이라는 단어가 얼마나 자주 발생합니까?

In [91]:
vectorizer = TfidfVectorizer(max_features = 15) # Top 15 terms
x = vectorizer.fit_transform(train['dialogue'])
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Unigrams', 'Train - Dialogue', 800, 800, 12)

이들 용어 사이의 상관관계는 매우 긍정적이지도, 매우 부정적이지도 않음을 알 수 있습니다. 가장 양의 상관관계가 있는 용어는 0.12에서 "don"과 "know"입니다. TfidfVectorizer 함수가 축약 제거와 같이 텍스트에 일부 변경을 수행하는 것을 관찰하는 것이 관련됩니다. 이는 아포스트로피 't 없이 단어가 표시되지 않는 이유를 설명합니다.

"예"와 "예"라는 용어 사이에 여전히 그다지 중요하지는 않지만 음의 상관관계가 있다는 점도 흥미롭습니다. 아마도 이는 동일한 대화에 두 가지를 모두 포함하는 것이 중복되기 때문에 발생할 수도 있고, 데이터가 대화 중에 "예" 대신 "예"를 사용하는 개인의 경향을 포착할 수도 있습니다. 이러한 유형의 히트맵을 분석할 때 고려할 수 있는 몇 가지 가설은 다음과 같습니다.

요약에 대해서도 동일한 분석을 수행해 보겠습니다.

In [92]:
vectorizer = TfidfVectorizer(max_features = 15) # Top 15 terms
x = vectorizer.fit_transform(train['summary'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Unigrams', 'Train - Summary', 800, 800, 12)

비록 이러한 상관관계가 여전히 강하지는 않지만 요약에 있는 용어의 상관관계는 대화에 있는 용어의 상관관계보다 더 뚜렷한 것 같습니다. 이는 요약이 전체 대화보다 관련 정보를 더 간결하게 전달할 수 있음을 시사하며, 이것이 바로 요약의 기본 아이디어입니다.

우리는 "가는 것"과 "만나다", "와서"와 "파티"뿐만 아니라 "구매하는 것"과 "원하는 것"과 같은 긍정적으로 상관된 쌍을 가지고 있습니다. 이러한 유니그램이 텍스트 전반에 걸쳐 함께 나타나는 것을 보는 것은 완벽하게 이해됩니다.

반대로, "going" 및 "wants", "going" 및 "got"과 같이 음의 상관 관계가 있는 쌍이 텍스트에서 자주 동시에 발생하지 않는 것이 합리적입니다.

이제 대화와 요약을 통해 바이그램을 분석해 보겠습니다.

In [93]:
vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (2,2)) # Top 15 terms
x = vectorizer.fit_transform(train['dialogue'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Bigrams', 'Train - Dialogue', 800, 800, 12)

다시 한번 말하지만, 상관관계는 그다지 강하지 않습니다. 그럼에도 불구하고 "있나요 Person1", "있습니다 person2"와 같이 함께 있으면 합리적으로 보이는 몇 가지 쌍을 볼 수 있습니다.

In [94]:

vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (2,2)) # Top 15 terms
x = vectorizer.fit_transform(train['summary'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Bigrams', 'Train - Summary', 800, 800, 12)

"person2 person1"와 "person1 에게" 쌍 사이에는 단 하나의 상관관계가 있습니다. 다른 용어는 어떤 종류의 상관관계도 전혀 없는 것으로 보입니다.

대화에는 없는 것 같은 회의록 정보를 요약에 포함하는 경향을 보는 것은 흥미롭습니다. 요약에 바이그램 15분이 나타나는 일부 요약을 쿼리하여 이 관계를 더 자세히 조사할 수도 있습니다.

In [95]:
pd.set_option('display.max_colwidth', None)
# Filtering dataset to see those containing the term '15 minutes' in the summary
filtered_train = train[train['summary'].str.contains('요', case=False, na=False)]
filtered_train.head()

,fname,dialogue,summary,topic
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 나와 함께 해줄래?\n#Person2#: 그는 귀여워! 그는 타이거 우즈 같아 보여! 하지만, 나는 춤을 못 춰...\n#Person1#: 괜찮아. 내가 너에게 모든 올바른 동작을 보여줄게. 내 이름은 말릭이야.\n#Person2#: 만나서 반가워. 나는 웬이고, 이쪽은 니키야.\n#Person1#: 기분 어때, 비스타? 네 친구를 춤추는 곳으로 데려가도 괜찮을까?\n#Person2#: 네 발을 밟아도 상관없다면 그녀는 신경 쓰지 않을 거야.\n#Person1#: 좋아. 멋져! 가자!",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 동의한다.,댄스
6,train_6,"#Person1#: 여기 도착했습니다.\n#Person2#: 감사합니다. 요금은 얼마인가요?\n#Person1#: 10달러입니다.\n#Person2#: 어떻게 그럴 수 있죠?\n#Person1#: 첫 2킬로미터는 2달러이고, 그 이후 200미터마다 20센트가 추가됩니다.\n#Person2#: 알겠습니다. 운전해주셔서 감사합니다.",#Person1#이 #Person2#에게 택시 요금을 알려줍니다.,요금 지불하기
7,train_7,"#Person1#: 도와드릴 일이 있나요?\n#Person2#: 네. 저는 지난 주말에 이력서를 보냈습니다. 회계 보조 자리에 지원하려고 합니다. \n#Person1#: 이름을 알려주실 수 있나요?\n#Person2#: 제 이름은 주디 리아오입니다. 엘 아이 에이 오로 씁니다. \n#Person1#: 알겠습니다. . . 지원서에 대해 특별히 궁금한 점이 있나요?\n#Person2#: 아니요. 근처에 있어서 제 이력서를 받았는지 확인하러 들렸습니다.\n#Person1#: 아, 문제 없습니다. 잠시만 기다려주시면 확인해보겠습니다. 주디 리아오. 찾아보겠습니다. . . 네, 여기 있네요.주디 리아오. 이력서를 받았습니다.\n#Person2#: 감사합니다.\n#Person1#: 다른 도움이 필요한 것이 있나요?\n#Person2#: 네, 아마도요. 신문 광고에는 이력서, 자기소개서, 그리고 추천서 두 통을 원한다고 써 있었습니다. 그것들을 모두 봉투에 넣어 보냈습니다. 다른 것을 보내야 할까요?\n#Person1#: 아니요, 그것들만 있으면 충분합니다. 그것들이 포함되어 있다면 충분합니다.\n#Person2#: 면접 일정이 언제 시작되는지 아시나요?\n#Person1#: 그건 정확히 모르겠습니다. 하지만 아직도 이력서를 받고 있습니다. 아마 일주일이나 두 주 후에 지원자들에게 전화를 걸기 시작할 것 같습니다.\n#Person2#: 알겠습니다. 도와주셔서 정말 감사합니다. 많은 도움이 되었습니다.\n#Person1#: 추가적인 질문이 있으시면 언제든지 전화하실 수 있습니다.\n#Person2#: 감사합니다.\n#Person1#: 감사합니다. 안녕히 가세요.","주디 리아오는 회계 보조 자리에 지원하고 있습니다. 그녀는 #Person1#에게 이력서를 받았는지 물었고, #Person1#은 그녀에게 확인해주었습니다. #Person1#은 주디에게 다른 것을 보낼 필요가 없다고 말하고, 일주일이나 두 주 후에 지원자들에게 전화를 걸기 시작할 것이라고 말했습니다.",직장 지원
11,train_11,"#Person1#: 봐! 학사모와 가운을 입고 있는 엄마 사진이야.\n#Person2#: 정말 멋지지 않아! 그때 엄마가 마이애미 대학교에서 석사 학위를 받았던 때야.\n#Person1#: 그래, 우리는 그녀를 정말 자랑스럽게 생각해.\n#Person2#: 오, 너희 모두가 함께 있는 이 사진도 좋네. 네거티브 필름 가지고 있니? 복사본을 가질 수 있을까?\n#Person1#: 물론이지, 너를 위해 하나 만들어 줄게. 프린트를 원해?\n#Person2#: 아니. 슬라이드를 원해, 새로운 프로젝터를 가지고 있거든.\n#Person1#: 나도 그걸 보고 싶네.\n#Person2#: 나를 위해 지갑 사이즈의 프린트도 만들어 줘.\n#Person1#: 당연하지.",#Person2#는 사진이 아름답다고 생각하고 #Person1#에게 슬라이드와 지갑 사이즈의 프린트를 달라고 요청한다.,복사본
22,train_22,"#Person1#: 제가 들었는데, 여러분들이 아주 좋은 서비스를 제공한다고 해서 이사할 때 필요한 이사업체로 먼저 연락했습니다.\n#Person2#: 저희에게 연락해주셔서 감사합니다. 어떤 일을 저희에게 부탁하실지 좀 더 자세히 말씀해주실 수 있을까요?\n#Person1#: 아, 저희는 8층에 있고, 다른 건물의 6층으로 이사를 가려고 합니다. 거기까지는 대략 15킬로미터 정도입니다.\n#Person2#: 알겠습니다, 비용은 이사할 층수, 두 장소 사이의 거리, 그리고 이동할 가구의 양에 따라 달라집니다.\n#Person1#: 그럼 그 경우에 비용이 얼마나 들까요?\n#Person2#: 아, 잠시만요. 이것은 두 번째 표준 요금에 해당합니다. 계약서를 한번 보시겠어요?\n#Person1#: 요금은 선불금과 나머지로 나뉘어져 있네요. 저는 이사하기 전에 전부 내야 한다고 생각했어요.\n#Person2#: 아니요, 먼저 계약서를 작성하고 ; 비용의 50%를 지불하시고, 나머지는 이사가 끝난 후에 지불하시면 됩니다.\n#Person1#: 손상 및 보상 항목이 헷갈리네요. 설명 좀 해주실 수 있을까요?\n#Person2#: 알겠습니다. 이사 중에 물품이 손상되었다면, 저희 부서에 보상 청구를 할 수 있습니다.",#Person1#은 이사업체가 필요해서 #Person2#에게 전화를 합니다. #Person1#은 #Person2#에게 필요한 일을 말하고 비용을 묻습니다. #Person2#은 계약서를 보여주고 비용과 보상을 설명합니다.,이사업체


마지막 행은 요약에서는 분과 관련된 용어가 그렇게 많이 보이지만 대화에서는 볼 수 없는 이유에 대한 아이디어를 제공합니다. 대화에서 사람들은 "것을 제안드립니다"과 같은 다른 형태로 쓸 수 있는 반면, 요약은 패턴화된 설명을 제공하므로 시간을 설명하기 위해 다른 형태보다 더 눈에 띄는 것이 자연스럽습니다.

이제 트라이그램을 시각화해 보겠습니다.

In [96]:
vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (3,3)) # Top 15 terms
x = vectorizer.fit_transform(train['dialogue'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Trigrams', 'Train - Dialogue', 800, 800, 12)

In [97]:
vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (3,3)) # Top 15 terms
x = vectorizer.fit_transform(train['summary'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Trigrams', 'Train - Summary', 800, 800, 12)

다시 한번, 우리는 용어들이 강한 상관관계가 없음을 알 수 있습니다. 그러나 여전히 논리적으로 보이는 쌍이 코퍼스에 함께 나타나는 것을 볼 수 있습니다.

이제 테스트 및 Val 데이터 세트에 대해 정확히 동일한 분석을 수행하겠습니다. 훈련 세트를 분석하는 동안 나타난 것과 동일한 동작을 기대할 수 있으므로 중복을 피하기 위해 다음 플롯에 대한 언급은 자제하겠습니다. 하지만, 다른 점이 나타나면 반드시 추가 조사를 하도록 하겠습니다.

# Test Dataset

In [98]:
describe_df(test)


DataFrame shape: (499, 2)

499 samples

2 attributes

Missing Data: 
fname       0
dialogue    0
dtype: int64

Duplicates: 0

Data Types: 
fname       object
dialogue    object
dtype: object

Categorical Features: 
fname, dialogue

Continuous Features: 
None

Binary Features: 
None

DataFrame Head: 



,fname,dialogue
0,test_0,"#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. \n#Person2#: 네, 실장님...\n#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?\n#Person2#: 네, 실장님. 시작하셔도 됩니다.\n#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.\n#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?\n#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.\n#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.\n#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?\n#Person2#: 이것은 내부와 외부 통신에 적용됩니다.\n#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.\n#Person2#: 그게 다신가요?\n#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요."
1,test_1,"#Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?\n#Person2#: 또 교통 체증에 걸렸어. 까르푸 교차로에서 엄청난 교통 체증이 있었어.\n#Person1#: 그곳은 출퇴근 시간에 항상 교통이 많이 밀리는 편이야. 집에 가는 다른 길을 찾아 보도록 해. \n#Person2#: 솔직히 말하면 피할 수 없는 것 같아.\n#Person1#: 아마 너가 대중교통을 이용하기 시작한다면 더 좋을 것 같아.\n#Person2#: 그건 고려해봐야 할 것 같아. 대중교통 시스템이 꽤 괜찮아.\n#Person1#: 환경에도 더 좋을 거야.\n#Person2#: 알아. 내 차가 이 도시의 오염 문제에 얼마나 기여하고 있는지 나도 느낄 수 있어.\n#Person1#: 지하철을 타는 것이 운전하는 것보다 훨씬 스트레스를 덜 받을 수 있을 거야.\n#Person2#: 유일한 문제는 차를 가지고 있는 자유를 정말로 그리워할 것 같다는 거야.\n#Person1#: 그럼, 날씨가 좋을 때는 자전거로 출근해봐. 그게 너의 차가 주는 자유와 같을 거야.\n#Person2#: 그게 맞는 말이야. 운동도 좀 해야겠어!\n#Person1#: 그럼, 너 출근할 때 운전하는 걸 그만두려고 해?\n#Person2#: 그래, 나에게도 환경에게도 좋지 않아."
2,test_2,"#Person1#: 케이트, 무슨 일이 일어났는지 너는 믿지 못할거야. \n#Person2#: 어떤 말이야?\n#Person1#: 마샤와 히어로가 이혼하려고 해.\n#Person2#: 너 농담하는 거지. 무슨 일이 있었어?\n#Person1#: 음, 정확히는 모르겠지만, 그들이 2개월 동안 별거 중이다가 이혼을 신청했다고 들었어.\n#Person2#: 정말 놀랍다. 나는 항상 그들이 잘 어울린다고 생각했어. 아이들은 어떻게 되는 거야? 어느 쪽이 양육권을 가지게 되는 거야?\n#Person1#: 마샤가 양육권을 가지게 되는 것 같아, 집과 주식을 누가 소유하는지에 대한 다툼도 없었고, 다른 세부 사항을 가지고 이혼에 이의를 제기하는 일도 없었대.\n#Person2#: 우리가 평소에 듣던 이혼 소식과는 다르네. 그래도, 여전히 믿기지 않아, 마샤와 히어로, 완벽한 커플이었는데. 이혼이 최종적으로 언제 확정되는 거야?\n#Person1#: 새해 초에 확정될 것 같아."
3,test_3,"#Person1#: 생일 축하해, 이건 너를 위한 거야, 브라이언.\n#Person2#: 기억해줘서 너무 행복해, 들어와서 파티를 즐겨. 모두 여기 있어, 분명히 너도 즐거울 거야.\n#Person1#: 브라이언, 너와 춤을 추는 기쁨을 가져도 될까?\n#Person2#: 좋아.\n#Person1#: 정말 멋진 파티야.\n#Person2#: 그래, 넌 항상 모두에게 인기가 많아. 그리고 오늘 너는 정말 예뻐 보여.\n#Person1#: 고마워, 그런 말 해줘서. 내 목걸이가 드레스와 잘 어울리길 바라. 둘 다 나를 잘 보이게 해주는 것 같아.\n#Person2#: 넌 멋져, 정말로 빛나고 있어.\n#Person1#: 고마워, 이건 정말 좋은 파티야. 너의 생일을 축하하기 위해 함께 술 한 잔 해야겠어"
4,test_4,"#Person1#: 이 올림픽 공원이 정말 크네요!\n#Person2#: 네. 지금 우리는 이 공원의 중심인 올림픽 스타디움에 있어요. \n#Person1#: 멋지네요! 언제 완공될 예정인가요?\n#Person2#: 이 스타디움은 6월에 완공될 예정이예요. \n#Person1#: 관람석에는 몇 개의 좌석이 있나요?\n#Person2#: 아, 총 5000개의 좌석이 있어요. \n#Person1#: 이렇게 크다니 몰랐어요!\n#Person2#: 그렇습니다! 저기 보세요, 그것들이 트랙이에요. 그리고 점프 피트는 저쪽에 있어요.\n#Person1#: 아... 알겠습니다. 저기요, 이 표지판을 보세요, 등반 금지예요. \n#Person2#: 우리는 외국인 방문객을 위해 영어로 번역된 표지판을 많이 설치했어요."



DataFrame Tail: 



,fname,dialogue
494,test_495,"#Person1#: 헤이, 찰리, 학교 끝나고 우리 집에 와서 나랑 비디오 게임 할래? 방금 멋진 새 게임이 생겼어. 6시에 공항으로 아빠를 데리러 가야하지만 2시간 동안 머물 수 있어.\n#Person2#: 그래, 잭. 숙제만 먼저 끝내면 돼. 어떤 게임을 얻었어?\n#Person1#: 네가 자신의 캐릭터를 만드는 게임이야. 너는 어떻게 생겼는지, 어떻게 입고 행동하는지를 선택할 수 있어.\n#Person2#: 흥미롭네. 그런 건 해본적이 없어. 정말 색다르게 들린다.\n#Person1#: 그래. 꼭 와줘. 빨리 보여주고 싶어."
495,test_496,"#Person1#: 어떻게 컨트리 음악에 관심을 가지게 되었나요?\r\n#Person2#: 아, 제 아내와 제가 처음으로 레코드 플레이어를 구입했을 때였습니다. 우리는 모든 종류의 레코드를 사기 시작했고, 곧, 저는 다른 어떤 종류보다도 컨트리 음악 레코드를 더 많이 사고 있다는 것을 발견했습니다.\r\n#Person1#: 어떻게 라디오 방송국에서 일하기 시작했나요?\r\n#Person2#: 컨트리 음악의 라디오 프로그램이 있어야 한다고 생각해서 CBC에 가서 제안했습니다. 그게 우리가 '골든 컨트리 타임'이라는 프로그램을 시작하게 된 계기였습니다.\r\n#Person1#: 그런 다음 그 노래들의 배경을 설명하는 기사를 쓰라는 요청을 받았나요.\r\n#Person2#: 네, 그런데 곧 정보를 찾기 위해 도서관에 달려가는 것에 지쳤습니다. 그래서 컨트리 음악 책을 사기 시작하고 제 자신의 도서관을 만들기 시작했습니다."
496,test_497,"#Person1#: 실례합니다, 앨리스. 이곳을 사용해본 적이 없는데, 기계를 어떻게 사용하는지 알려주실 수 있나요?\r\n#Person2#: 어떤 말씀이신가요? 이것들은 세탁기입니다. 저기 큰 것들은 건조기입니다.\r\n#Person1#: 알겠습니다. 기계 안에 비누가 있나요?\r\n#Person2#: 아니요, 당연히 없습니다. 비누를 넣어야 합니다. 비누를 가져오셨나요?\r\n#Person1#: 아니요, 비누가 없습니다.\r\n#Person2#: 그럼, 저기서 사실 수 있습니다.\r\n#Person1#: 감사합니다. 비누를 샀습니다.\r\n#Person2#: 이런, 정말 많이 사셨네요. 왜 이렇게 많이 필요한가요?\r\n#Person1#: 잘 모르겠습니다. 옷을 깨끗하게 하고 싶습니다.\r\n#Person2#: 하지만 이렇게 많이 사용하면 안 됩니다. 기계가 비누를 완전히 제거하지 못합니다. 오히려 너무 많은 거품이 먼지를 가둬서, 세균이 쌓이게 됩니다.\r\n#Person1#: 아, 그렇군요. 몰랐습니다. 저는 옷을 빨아본 적이 없습니다.\r\n#Person2#: 뭐요? 옷을 빨아본 적이 없다고요?\r\n#Person1#: 네, 한 번도 없습니다.\r\n#Person2#: 믿을 수 없네요. 어떻게 그럴 수 있나요?\r\n#Person1#: 제 엄마가 항상 해주셨습니다.\r\n#Person2#: 네, 제 엄마도 제 옷을 빨아주셨습니다. 하지만 12살 때부터 제가 직접 옷을 빨기 시작했습니다.\r\n#Person1#: 그 사실은 알고 있습니다. 미국 아이들은 더 독립적입니다. 하지만 제 나라에서는 엄마가 아이들이 학교에서 좋은 성적을 받기를 원하기 때문에, 모든 것을 아이들 대신 해줍니다.\r\n#Person2#: 그렇다면 캠퍼스에서 어떻게 살아갈 건가요?\r\n#Person1#: 잘 모르겠습니다. 힘들지만 배워야 합니다."
497,test_498,"#Person1#: 매튜? 안녕!\r\n#Person2#: 스티브! 오랜만이네! 얼마나 됐지?\r\n#Person1#: 거의 1년이 되어가는 것 같아.\r\n#Person2#: 어떻게 지냈어?\r\n#Person1#: 꽤 괜찮아. 최근에 살 곳을 찾고 있어. 내 계약이 다음 달에 끝나는데, 갱신하고 싶지 않아.\r\n#Person2#: 응, 그 동네 기억해. 이미 집을 찾았어?\r\n#Person1#: 아직이야. 나는 여전히 분류된 광고를 찾고 있어. 행운을 빌어줘.\r\n#Person2#: 음, 아마 나도 도움이 될 수 있을 것 같아. 내 이웃을 기억하니?\r\n#Person1#: 다우 부인 말이야?\r\n#Person2#: 응, 그녀의 딸이 아기를 가지게 돼서, 그녀는 딸과 함께 살면서 돕기로 했어. 관심 있다면, 너도 한번 와서 둘러볼 수 있을 것 같아.\r\n#Person1#: 좋아. 정말 좋은 동네야. 그리고 다시 이웃이 될 수 있다면 좋겠어. 예전처럼 말이야!\r\n#Person2#: 다우 부인에게 언제 아파트를 보여줄 수 있는지 물어보고 알려줄게. 너의 전화번호가 바뀌었어?"
498,test_499,"#Person1#: 헤이, 벳시, 좋은 소식 들었어?\n#Person2#: 아니, 프랭크, 못 들었어. 무슨 일이야?\n#Person1#: 나 방금 승진했어. 그리고 친구들 모두를 위한 큰 파티를 열려고 해. 너도 와주면 좋겠어.\n#Person2#: 와우, 정말 고마워. 파티는 언제야?\n#Person1#: 토요일에 하려고 생각 중이야. 150명 정도 올 거라고 기대하고 있어.\n#Person2#: 와우, 정말 많네. 이건 분명히 큰 승진이겠네. 나도 가고 싶어. 재미있을 것 같아.\n#Person1#: 오, 좋아. 사람이 많을수록 좋아. 이건 정말로 나에게 큰 일이야. 이제 아내가 항상 원했던 새 집을 살 수 있게 됐어. 나는 단지 너무 많은 시간을 들일 필요가 없었으면 좋겠어. 가족과의 시간을 너무 많이 잃고 싶지 않아.\n#Person2#: 그건 이해해. 하지만 계속해서 긍정적인 면에 집중하자. 그 파티가 기다려지네."


In [99]:
df_text_lenght = pd.DataFrame()
for feat in categorical_features:
    df_text_lenght[feat] = test[feat].apply(lambda x: len(str(x).split()))

histogram_boxplot(df_text_lenght,'#89c2e0', '#d500ff', 600, 1000, True, 'Test Dataset')

An error occurred: The data appears to lie in a lower-dimensional subspace of the space in which it is expressed. This has resulted in a singular data covariance matrix, which cannot be treated using the algorithms implemented in `gaussian_kde`. Consider performing principle component analysis / dimensionality reduction and using `gaussian_kde` with the transformed data.


In [100]:
vectorizer = TfidfVectorizer(max_features = 15) # Top 15 terms
x = vectorizer.fit_transform(test['dialogue'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Unigrams', 'Test - Dialogue', 800, 800, 12)

In [101]:
vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (2,2)) # Top 15 terms
x = vectorizer.fit_transform(test['dialogue'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Bigrams', 'Test - Dialogue', 800, 800, 12)

In [102]:

vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (3,3)) # Top 15 terms
x = vectorizer.fit_transform(test['dialogue'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Trigrams', 'Test - Dialogue', 800, 800, 12)

# Val Dataset

In [103]:
describe_df(val)


DataFrame shape: (499, 4)

499 samples

4 attributes

Missing Data: 
fname       0
dialogue    0
summary     0
topic       0
dtype: int64

Duplicates: 0

Data Types: 
fname       object
dialogue    object
summary     object
topic       object
dtype: object

Categorical Features: 
fname, dialogue, summary, topic

Continuous Features: 
None

Binary Features: 
None

DataFrame Head: 



,fname,dialogue,summary,topic
0,dev_0,"#Person1#: 안녕하세요, 오늘 하루 어떠셨어요? \n#Person2#: 요즘 숨쉬기가 좀 힘들어요.\n#Person1#: 최근에 감기 같은 것에 걸리신 적이 있나요?\n#Person2#: 아니요, 감기는 아니에요. 그냥 숨을 쉴 때마다 가슴이 무겁게 느껴져요.\n#Person1#: 알고 있는 알레르기가 있나요?\n#Person2#: 아니요, 알고 있는 알레르기는 없어요.\n#Person1#: 이런 증상이 항상 나타나나요, 아니면 활동할 때 주로 나타나나요?\n#Person2#: 운동을 할 때 많이 나타나요.\n#Person1#: 저는 당신을 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 거예요.\n#Person2#: 도와주셔서 감사합니다, 의사 선생님.","#Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대해 묻고, #Person2#를 폐 전문의에게 보낼 예정이다.",의사에게 상담하기
1,dev_1,"#Person1#: 헤이, 지미. 나중에 운동하러 가자.\n#Person2#: 그래. 몇 시에 갈까?\n#Person1#: 3시 30분 어때?\n#Person2#: 좋아. 오늘은 다리와 팔목을 운동하자.\n#Person1#: 헤이. 나 방금 농구를 했는데, 다리가 좀 아파. 오늘은 팔과 배를 운동하자.\n#Person2#: 나는 주간 스케줄을 따르고 있어. 너 때문에 모든 게 망가지고 있어.\n#Person1#: 제발. 우리는 단지 두 날을 바꾸는 거야. 금요일에 다리를 할 수 있어.\n#Person2#: 알았어. 그럼 3시 30분에 헬스장에서 만나자.",#Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.,운동하기
2,dev_2,"#Person1#: 나는 더 이상 건강에 해로운 음식을 먹는 것을 멈춰야 해.\n#Person2#: 무슨 말인지 알아. 나도 요즘에는 더 건강한 음식을 먹기 시작했어.\n#Person1#: 지금은 어떤 음식을 먹고 있어?\n#Person2#: 나는 주로 과일, 채소, 그리고 닭고기를 먹는 편이야.\n#Person1#: 그것들만 먹는 거야?\n#Person2#: 그게 기본적으로 내가 먹는 것이야.\n#Person1#: 왜 다른 것은 먹지 않는 거야?\n#Person2#: 음, 과일과 채소는 매우 건강하거든.\n#Person1#: 그럼 닭고기는?\n#Person2#: 닭고기는 구워서 먹으면 정말 건강에 좋아.\n#Person1#: 그렇게 들어보니 훨씬 더 건강해 보이는군.","#Person1#은 건강에 해로운 음식을 먹는 것을 멈추려는 계획을 세우고, #Person2#는 자신의 건강한 레시피를 #Person1#와 공유한다.",건강한 음식
3,dev_3,"#Person1#: UFO를 믿으세요?\n#Person2#: 물론이죠, 그들은 저기 어딘가에 있어요.\n#Person1#: 하지만 저는 그들을 본 적이 없어요.\n#Person2#: 당신은 바본가요? 그들이 UFO라고 불리우는 이유가 모두가 볼 수 있는 것이 아니기 때문이에요.\n#Person1#: 그러니까 당신은 그들을 볼 수 있다는 거죠.\n#Person2#: 맞아요. 저는 꿈에서 그들을 볼 수 있어요.\n#Person1#: 그들이 지구에 오나요?\n#Person2#: 아니요. 그들의 임무는 외계인들을 우주에서 여기로 보내는 것이에요.\n#Person1#: 우주에선 온 외계인들이라고요? 그들과 대화하나요? 그들은 어떻게 생겼나요?\n#Person2#: 좋아요, 좋아요, 하나씩이요! 그들은 로봇처럼 생겼지만 말할 수 있어요. 그들의 임무는 인간과 친구가 되는 것이에요.\n#Person1#: 그러니까 당신이 그들과 대화한다는 거죠? 어떤 언어로요?\n#Person2#: 당연히 영어로요, 그들도 화성에서 영어를 배우거든요.\n#Person1#: 와. 정말 멋져 보이네요!","#Person2#는 UFO를 믿고 꿈에서 그들을 볼 수 있다고 말한다. #Person1#는 #Person2#에게 UFO와 꿈 속의 외계인에 대해 묻고, #Person2#의 꿈을 멋지다고 느낀다.",UFO와 외계인
4,dev_4,"#Person1#: 오늘 학교에 갔어?\n#Person2#: 당연하지. 너는?\n#Person1#: 나는 가고 싶지 않아서 안 갔어.\n#Person2#: 그게 슬프네, 그런데 최근에 영화 보러 갔어?\n#Person1#: 갑자기 그런 얘기를 하네.\n#Person2#: 진심이야, 갔어?\n#Person1#: 아니, 안 갔어. 왜?\n#Person2#: 나 이번 주말에 정말로 영화 보러 가고 싶어.\n#Person1#: 그럼 가면 되지.\n#Person2#: 혼자 가고 싶지 않아.\n#Person1#: 그래도 어쨌뜬, 내일은 학교에 갈 계획이지?\n#Person2#: 아니, 나는 영화 보러 갈 것 같아.",#Person1#은 오늘 학교에 가지 않았다. #Person2#는 내일 수업을 빼먹고 영화 보러 가려고 한다.,학교 가기



DataFrame Tail: 



,fname,dialogue,summary,topic
494,dev_495,"#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#Person2#: 그래? 드디어 모든 것을 새롭게 시작하려고 결정했어? \r\n#Person1#: 맞아! 새로운 일자리가 있고, 새로운 도시에서 살고, 새로운 친구들이 있어! 이것이 내 생활 방식에 약간의 변화를 줄 기회야. \r\n#Person2#: 그럼 뭘 할 거야? 미술 수업 같은 걸 들을 거야? \r\n#Person1#: 우선, 금연을 하기로 결심했어. 돈을 아끼려는 건 아니고, 16살 때부터 흡연을 해 왔는데, 이제 그만둘 시간이라고 생각해. \r\n#Person2#: 그건 나도 동감이야. 다른 계획은 뭐가 있어? \r\n#Person1#: 마지막으로, 나는 커밍아웃하기로 결정했어. \r\n#Person2#: 드디어 그렇게 결정했구나!",#Person1#은 새해에 금연을 하고 커밍아웃하기로 결정했습니다. #Person2#는 #Person1#를 지지합니다.,새해
495,dev_496,"#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인지 모르겠어. \r\n#Person1#: 네가 사랑에 빠졌다는 걸 기억해. \r\n#Person2#: 아, 처음 만났을 때 그에게 호감을 가졌었어. 하지만 마음이 갈팡질팡해.\r\n#Person1#: 무슨 말인지 전혀 모르겠네.",#Person1#은 #Person2#가 조와 결혼했다고 생각했다. #Person2#는 부인했다.,사랑에 빠지다
496,dev_497,"#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동안 차에서 이상한 소리가 나서 오늘 한번 봐주실 수 있을까요?\r\n#Person1#: 어떤 종류의 소리인가요?\r\n#Person2#: 바퀴에서 뭔가가 손상되는 것 같은 소리가 나요. 제가 속도를 줄일 때만 그런 소리가 나요.\r\n#Person1#: 와우, 브레이크를 새로 교체해야 할 것 같네요. 차를 내일까지 저희에게 맡겨야 할 것 같아요.\r\n#Person2#: 이런, 오늘 오후에 차를 돌려받을 수 있을 줄 알았어요.\r\n#Person1#: 유감스럽게도, 부품을 주문해야 하고 그것들이 도착하기 전에는 작업을 시작할 수 없어요. 지금 주문하면 오후나 내일 아침에는 도착할 것입니다.\r\n#Person2#: 알겠습니다. 그럼 내일 아침에 차를 다시 가져오는 게 어떨까요? 오늘 밤에 꼭 보고 싶은 공연이 시내에 있거든요.\r\n#Person1#: 그건 좋은 생각이 아닌 것 같아요. 당신은 이 차를 운전하면서 목숨을 걸고 있어요. 제가 당신이라면 버스 시간표를 확인해 볼 것입니다.","#Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교체해야 할 것으로 생각하지만, #Person1#는 내일까지 그것을 고칠 수 없습니다. #Person2#는 오늘 밤에 공연을 보러 가고 싶어합니다; #Person1#는 #Person2#에게 버스를 이용할 것을 제안합니다.",소음
497,dev_498,"#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n#Person2#: 안녕하세요, 어제 웹사이트에서 받은 책을 읽다가 한 페이지가 빠져있는 것을 발견했습니다, 53페이지에요.\n#Person1#: 알겠습니다. 주문 번호를 알려주실 수 있을까요, 고객님?\n#Person2#: B113-7423935입니다.\n#Person1#: 확인해보겠습니다. 10일 전인 10월 13일에 웹사이트에서 구매하신 라 살바토레의 '헌터의 종이 봉투 밤'이 맞나요?\n#Person2#: 네.\n#Person1#: 그럼, 이 책의 빠진 부분을 사진으로 찍어서 우리 웹사이트의 고객 서비스 페이지에 업로드해주셔야 할 것 같습니다. 문제가 확인되면 2일 내로 새 책을 보내드리겠습니다.\n#Person2#: 알겠습니다. 그럼 이전에 받은 책은 어떻게 해야 하나요? 돌려보내야 하나요?\n#Person1#: 그럴 필요는 없습니다, 고객님. 원하신다면 그냥 가지고 계셔도 됩니다. 다른 도움이 필요한 것이 있으신가요?\n#Person2#: 아니요, 감사합니다.\n#Person1#: 천만에요, 고객님. 좋은 하루 보내세요.",#Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지가 빠져 있다고 합니다. #Person1#은 문제가 확인되면 새 책을 보내드릴 것이라고 말합니다.,빠진 페이지
498,dev_499,"#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#: 응, 알아. 이번 해는 정말 빨리 갔어.\r\n#Person1#: 이번 여름 휴가에 뭐 할 거야?\r\n#Person2#: 나는 회사에서 일할 거야.\r\n#Person1#: 회사? 그게 뭐야? 무슨 일을 하는 거야?\r\n#Person2#: 우리는 파티를 도와. 우리 회사는 음식을 준비하고 제공하고, 보통 다른 회사에서 음악을 제공해.\r\n#Person1#: 네가 요리를 할 줄 몰랐어.\r\n#Person2#: 나는 요리를 할 필요가 없어. 나는 그저 조수일 뿐이야.\r\n#Person1#: 언제 시작해?\r\n#Person2#: 내일이야. 우리는 생일 파티를 도와줄 거야. 그리고 대가족이 모일거야.\r\n#Person1#: 그럼, 너는 정확히 뭘 하는 거야?\r\n#Person2#: 파티가 시작하기 전에, 나는 모든 것을 준비하는 것을 도와. 음식을 가져다 놓고, 테이블을 정리하고, 예쁘게 보이게 하는 거지.\r\n#Person1#: 꽤 쉬워 보이네.\r\n#Person2#: 그건 일부일 뿐이야. 파티 도중에는 손님들에게 음식과 음료를 제공해야 해.\r\n#Person1#: 그래도 사람들을 만날 수 있어서 좋겠다.\r\n#Person2#: 응, 그리고 파티가 끝나면 청소를 도와.\r\n#Person1#: 윽, 설거지하는 건 싫어.\r\n#Person2#: 아, 나는 설거지를 안 해. 다른 사람이 해. 나는 그냥 모든 것을 트럭에 넣을 뿐이야.\r\n#Person1#: 그렇게 나쁘지 않네. 꽤 멋진 직업 같아.",#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 일할 것이라고 말한다. #Person1#는 그것이 멋진 직업이라고 생각한다.,여름 휴가


In [104]:
df_text_lenght = pd.DataFrame()
for feat in categorical_features:
    df_text_lenght[feat] = val[feat].apply(lambda x: len(str(x).split()))

histogram_boxplot(df_text_lenght,'#89c2e0', '#d500ff', 600, 1000, True, 'Validation Dataset')

An error occurred: The data appears to lie in a lower-dimensional subspace of the space in which it is expressed. This has resulted in a singular data covariance matrix, which cannot be treated using the algorithms implemented in `gaussian_kde`. Consider performing principle component analysis / dimensionality reduction and using `gaussian_kde` with the transformed data.


In [105]:
vectorizer = TfidfVectorizer(max_features = 15) # Top 15 terms
x = vectorizer.fit_transform(val['dialogue'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Unigrams', 'Validation - Dialogue', 800, 800, 12)

In [106]:
vectorizer = TfidfVectorizer(max_features = 15) # Top 15 terms
x = vectorizer.fit_transform(val['summary'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Unigrams', 'Validation - Summary', 800, 800, 12)

In [107]:
vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (2,2)) # Top 15 terms
x = vectorizer.fit_transform(val['dialogue'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Bigrams', 'Validation - Dialogue', 800, 800, 12)


In [108]:
vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (2,2)) # Top 15 terms
x = vectorizer.fit_transform(val['summary'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Bigrams', 'Validation - Summary', 800, 800, 12)

In [109]:
vectorizer = TfidfVectorizer(max_features = 15,ngram_range = (3,3)) # Top 15 terms
x = vectorizer.fit_transform(val['dialogue'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Trigrams', 'Validation - Dialogue', 800, 800, 12)


In [110]:
vectorizer = TfidfVectorizer(max_features = 15,stop_words = 'english',ngram_range = (3,3)) # Top 15 terms
x = vectorizer.fit_transform(val['summary'].fillna(''))
df_tfidfvect = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())
plot_correlation(df_tfidfvect, 'Trigrams', 'Validation - Summary', 800, 800, 12)


전반적으로 세 가지 데이터 세트 모두에서 유사한 패턴을 나타냅니다. 요약은 예상대로 대화보다 길이가 짧으며, 함께 사용하기에 합당해 보이는 많은 용어는 더 높은 수준의 상관관계를 갖습니다.

n-gram 히트맵을 분석하면 일반적으로 대화에 나타나는 많은 용어를 볼 수 있으므로 이 데이터가 채팅/대화 텍스트로 구성되어 있음이 분명해집니다.

In [112]:
# "요"를 포함하는 단어를 찾기 위한 정규표현식 패턴
pattern = re.compile(r'\b\w*요\w*\b')

# "dialogue" 열의 각 문장에서 "요"를 포함하는 단어를 찾습니다.
words_with_yo = []
for sentence in train['dialogue']:
    words_with_yo.extend(pattern.findall(sentence))

# 중복 제거
words_with_yo = list(set(words_with_yo))

print(words_with_yo)

['것이잖아요', '마셨는데요', '금요일이죠', '다녀왔나요', '유행이잖아요', '주소로요', '금요일부터', '않다구요', '드셨겠어요', '처리하려고요', '못했거든요', '다요', '시작했네요', '뜻이거든요', '복사했나요', '심했어요', '겪었어요', '귀여워요', 'BLC에요', '보내주었나요', '걱정돼요', '절망적이에요', '플레이해요', '원인이군요', '가야해요', '여쭤보려고요', '보내야겠어요', '받았었어요', '이체하시겠어요', '가져가주세요', '아빠니까요', '남네요', '대화하시나요', '보이거든요', '유동적이에요', '공부해야겠어요', '포기해요', '기다릴게요', '뵐게요', '나오더라고요', '요일인가요', '전데요', '다니시잖아요', '기지거든요', '문제군요', '브래드에요', '않으셨군요', '복사본이요', '학과이신가요', '들릴게요', '토요일이니까', '처음이지요', '관용구인가요', '연해요', '목적이에요', '뛰어야겠네요', '설명해야겠네요', '계좌요', '스타네요', '형성하나요', '가봐야겠어요', '시예요', '증명하나요', '두려우신가요', '준비됐나요', '기억해야겠어요', '도전적이에요', '개업했고요', 'RBM이요', '없으신가요', '최악이에요', '발생시켰나요', '괜찮을까요', '내려주시겠어요', '좋아하시나봐요', '지시했나요', '호심정이에요', '요가는', '식당이에요', '다음엔요', '좋군요', '효율적이네요', '15명이에요', '존중하세요', '아팠어요', '모두요', '못하셔요', '금요일쯤', '생신인가요', '같잖아요', '날씨에는요', '1파운드예요', '왔잖아요', '정원이네요', '돌아가시는군요', '일요일요', '변화무쌍해요', '시켜요', '말라고요', '2시쯤이었어요', '애나처럼요', '운영해봤어요', '소식이네요', '오늘은요', '높겠어요', '즐겨봐요', '쏟아버렸어요', '엄격해요', '부드럽군요', '증가시켜요', '일요일판이', '못할까

# Preprocessing Data

BART와 같은 사전 훈련된 모델을 사용하여 작업할 때의 주요 이점 중 하나는 이러한 모델이 일반적으로 매우 강력하고 데이터 사전 처리가 거의 필요하지 않다는 것입니다.

EDA를 수행하는 동안 file_photo와 같은 몇 가지 텍스트에 태그가 있다는 것을 발견했습니다. 몇 가지 예를 살펴보겠습니다.

Theresa: <file_photo>  
Theresa: <file_photo>  
Theresa: Hey Louise, how are u?  
Theresa: This is my workplace, they always give us so much food here 😊  
Theresa: Luckily they also offer us yoga classes, so all the food isn't much of a problem 😂  
Louise: Hey!! 🙂   
Louise: Wow, that's awesome, seems great 😎 Haha  
Louise: I'm good! Are you coming to visit Stockholm this summer? 🙂  
Theresa: I don't think so :/ I need to prepare for Uni.. I will probably attend a few lessons this winter  
Louise: Nice! Do you already know which classes you will attend?  
Theresa: Yes, it will be psychology :) I want to complete a few modules that I missed :)  
Louise: Very good! Is it at the Uni in Prague?  
Theresa: No, it will be in my home town :)  
Louise: I have so much work right now, but I will continue to work until the end of summer, then I'm also back to Uni, on the 26th September!  
Theresa: You must send me some pictures, so I can see where you live :)   
Louise: I will, and of my cat and dog too 🤗  
Theresa: Yeeeesss pls :)))  
Louise: 👌👌  
Theresa: 🐱💕  

아래에 정의된 clean_tags 함수를 사용하여 텍스트에서 이러한 태그를 제거하여 더 깔끔하게 만들겠습니다.

In [41]:
# def clean_tags(text):
#     clean = re.compile('<.*?>') # Compiling tags
#     clean = re.sub(clean, '', text) # Replacing tags text by an empty string

#     # Removing empty dialogues
#     clean = '\n'.join([line for line in clean.split('\n') if not re.match('.*:\s*$', line)])

#     return clean

In [42]:
# test1 = clean_tags(train['dialogue'].iloc[14727]) # Applying function to example text
# test2 = clean_tags(test['dialogue'].iloc[0]) # Applying function to example text

# # Printing results
# print(test1)
# print('\n' *3)
# print(test2)

Theresa: Hey Louise, how are u?  
Theresa: This is my workplace, they always give us so much food here 😊  
Theresa: Luckily they also offer us yoga classes, so all the food isn't much of a problem 😂  
Louise: Hey!! 🙂   
Louise: Wow, that's awesome, seems great 😎 Haha  
Louise: I'm good! Are you coming to visit Stockholm this summer? 🙂  
Theresa: I don't think so :/ I need to prepare for Uni.. I will probably attend a few lessons this winter  
Louise: Nice! Do you already know which classes you will attend?  
Theresa: Yes, it will be psychology :) I want to complete a few modules that I missed :)  
Louise: Very good! Is it at the Uni in Prague?  
Theresa: No, it will be in my home town :)  
Louise: I have so much work right now, but I will continue to work until the end of summer, then I'm also back to Uni, on the 26th September!  
Theresa: You must send me some pictures, so I can see where you live :)   
Louise: I will, and of my cat and dog too 🤗  
Theresa: Yeeeesss pls :)))  
Louise: 👌👌  
Theresa: 🐱💕  

텍스트에서 태그가 성공적으로 제거되었음을 확인할 수 있습니다. 이제 전체 데이터 세트에 clean_tags를 적용하는 clean_df 함수를 정의하겠습니다.

In [43]:
# # Defining function to clean every text in the dataset.
# def clean_df(df, cols):
#     for col in cols:
#         df[col] = df[col].fillna('').apply(clean_tags)
#     return df

In [44]:
# # Cleaning texts in all datasets
# train = clean_df(train,['dialogue', 'summary'])
# test = clean_df(test,['dialogue'])
# val = clean_df(val,['dialogue', 'summary'])

태그가 텍스트에서 제거되었습니다. 이러한 데이터 정리를 수행하여 노이즈(전체 컨텍스트에 크게 기여하지 않고 잠재적으로 성능을 저하시킬 수 있는 정보)를 제거하는 것이 좋습니다.

이제 사전 훈련된 모델에 대한 입력으로 사용하고 미세 조정을 위해 데이터를 준비하는 데 필요한 몇 가지 전처리를 수행하겠습니다. 제가 여기서 하고 있는 대부분의 작업은 여기에서 볼 수 있는 🤗 Transformers 문서에 설명된 텍스트 요약 튜토리얼의 일부입니다.

먼저 🤗 Datasets 라이브러리를 사용하여 Pandas Dataframe을 Dataset으로 변환하겠습니다. 이를 통해 Hugging Face 생태계 전체에서 데이터를 처리할 수 있게 됩니다.

In [45]:
# # Transforming dataframes into datasets
# train_ds = Dataset.from_pandas(train)
# test_ds = Dataset.from_pandas(test)
# val_ds = Dataset.from_pandas(val)

# # Visualizing results
# print(train_ds)
# print('\n' * 2)
# print(test_ds)
# print('\n' * 2)
# print(val_ds)

Dataset({
    features: ['fname', 'dialogue', 'summary', 'topic'],
    num_rows: 12405
})



Dataset({
    features: ['fname', 'dialogue'],
    num_rows: 499
})



Dataset({
    features: ['fname', 'dialogue', 'summary', 'topic'],
    num_rows: 499
})


🤗 데이터 세트 내부의 내용을 보려면 아래와 같이 특정 행을 선택하면 됩니다.

In [46]:
# train_ds[0] # Visualizing the first row

{'fname': 'train_0',
 'dialogue': '#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.',
 'summary': '스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.',
 'topic': '건강검진 받기'}

이렇게 하면 원본 ID, 대화 내용 및 참조 요약을 볼 수 있습니다. __index_level_0__은 데이터에 아무것도 추가하지 않으며 추가로 제거됩니다.

Pandas 데이터프레임을 🤗Datasets로 성공적으로 변환한 후 모델링 프로세스를 진행할 수 있습니다.

# Modeling

이전에 언급했듯이 텍스트 요약을 위한 여러 뉴스 기사(facebook/bart-large-xsum)에 대해 훈련된 BART 버전을 미세 조정할 예정입니다.

뉴스 데이터에서 어떻게 작동하는지 보여주기 위해 요약 파이프라인을 로드하여 이 모델을 간략하게 설명하겠습니다.

In [47]:
# # Loading summarization pipeline with the bart-large-cnn model
# summarizer = pipeline('summarization', model = 'facebook/bart-large-xsum')

예를 들어, 2023년 10월 24일 CNN에 게재된 다음 뉴스 기사를 사용하겠습니다. 세계에서 가장 나이 많은 개인 Bobi가 31세의 나이로 사망했습니다. 이 기사는 제가 전달하고 있는 전혀 보이지 않는 뉴스 기사라는 점에 유의하세요. 모델이 어떻게 작동하는지 확인할 수 있습니다.

모델이 입력 텍스트에 있는 가장 관련성 높은 정보로 구성된 훨씬 짧은 텍스트를 정확하게 생성할 수 있음을 확인할 수 있습니다. 성공적인 요약입니다.

그러나 이 모델은 많은 대화 데이터가 아닌 CNN과 Daily Mail의 여러 뉴스 기사로 구성된 데이터세트를 중심으로 훈련되었습니다. 이것이 바로 SamSum 데이터 세트를 사용하여 미세 조정하려는 이유입니다.

facebook/bart-large-xsum 체크포인트를 사용하여 BartTokenizer 및 BartForConditionalGeneration을 로드해 보겠습니다.

In [48]:
# checkpoint = 'facebook/bart-large-xsum' # Model
# tokenizer = BartTokenizer.from_pretrained(checkpoint) # Loading Tokenizer

In [49]:
# model = BartForConditionalGeneration.from_pretrained(checkpoint) # Loading Model

모델의 아키텍처 아래를 인쇄할 수도 있습니다.

In [50]:
# print(model) # Visualizing model's architecture

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

모델이 인코더와 디코더로 구성되어 있음을 볼 수 있으며, 선형 레이어와 일반적인 ReLU 대신 GeLU를 사용하는 활성화 함수를 볼 수 있습니다.

출력 레이어인 lm_head를 관찰하는 것도 흥미롭습니다. 이는 이 모델이 어휘 크기(out_features=50264)로 출력을 생성하는 데 이상적이라는 것을 보여줍니다. 이는 이 아키텍처가 요약 작업과 기타 작업에 적합하다는 것을 보여줍니다. 예를 들어 번역 같은 것 말이죠.

이제 BART 모델에서 데이터를 읽을 수 있도록 데이터 세트를 전처리하고 BartTokenizer를 사용해야 합니다.

다음 preprocess_function은 🤗 Transformers 문서에서 직접 복사할 수 있으며 여러 NLP 작업에 대한 데이터를 전처리하는 데 유용합니다. 필요한 단계를 설명하여 데이터를 전처리하는 방법에 대해 좀 더 자세히 살펴보겠습니다.

• inputs = [doc for doc in example["dialogue"]]: 이 줄에서는 데이터세트의 모든 대화를 반복하고 이를 모델에 대한 입력으로 저장합니다.

• model_inputs = 토크나이저(입력, max_length=1024, truncation=True)
: 여기서는 토크나이저를 사용하여 입력 대화를 BART 모델에서 쉽게 이해할 수 있는 토큰으로 변환합니다. truncation=True 매개변수는 max_length 매개변수에 정의된 대로 모든 대화에 최대 1024개의 토큰이 포함되도록 보장합니다.

• labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True): 이 줄은 위와 매우 유사한 토큰화 프로세스를 수행합니다. 그러나 이번에는 요약인 대상 변수를 토큰화합니다. 또한 여기의 max_length는 128로 상당히 낮습니다. 이는 요약이 대화의 텍스트보다 훨씬 짧은 텍스트가 될 것으로 예상함을 의미합니다.

• model_inputs["labels"] = labels["input_ids"]: 이 줄은 본질적으로 토큰화된 입력과 함께 토큰화된 레이블을 전처리된 데이터세트에 추가합니다.

In [51]:
# def preprocess_function(examples):
#     inputs = [doc for doc in examples["dialogue"]]
#     model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

#     # Setup the tokenizer for targets
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples["summary"], max_length=128, truncation=True)

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [52]:
# val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fname     499 non-null    object
 1   dialogue  499 non-null    object
 2   summary   499 non-null    object
 3   topic     499 non-null    object
dtypes: object(4)
memory usage: 15.7+ KB


In [53]:
# # Applying preprocess_function to the datasets
# tokenized_train = train_ds.map(preprocess_function, batched=True,
#                                remove_columns=['fname', 'dialogue', 'topic','summary']) # Removing features

# # tokenized_test = test_ds.map(preprocess_function, batched=True,
# #                                remove_columns=['fname']) # Removing features

# tokenized_val = val_ds.map(preprocess_function, batched=True,
#                                remove_columns=['fname', 'dialogue', 'topic','summary']) # Removing features

# # Printing results
# print('\n' * 3)
# print('Preprocessed Training Dataset:\n')
# print(tokenized_train)
# # print('\n' * 2)
# # print('Preprocessed Test Dataset:\n')
# # print(tokenized_test)
# print('\n' * 2)
# print('Preprocessed Validation Dataset:\n')
# print(tokenized_val)

Map:   0%|          | 0/12405 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]





Preprocessed Training Dataset:

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 12405
})



Preprocessed Validation Dataset:

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 499
})


우리의 토큰화된 데이터 세트는 이제 input_ids, attention_mask 및 labels의 세 가지 기능으로만 구성됩니다. 사전 처리 기능이 데이터를 어떻게 변경했는지 자세히 조사하기 위해 토큰화된 열차 데이터 세트의 샘플을 인쇄해 보겠습니다.

In [54]:
# # Selecting a sample from the dataset
# sample = tokenized_train[0]

# # Printing its features
# print("input_ids:")
# print(sample['input_ids'])
# print("\n")
# print("attention_mask:")
# print(sample['attention_mask'])
# print("\n")
# print("sample:")
# print(sample['labels'])
# print("\n")

input_ids:
[0, 10431, 41761, 134, 10431, 35, 46747, 15722, 23133, 45209, 5782, 15722, 48589, 711, 43998, 11936, 18537, 43998, 15113, 10674, 6, 46747, 27969, 10470, 45209, 10965, 18537, 43998, 27969, 10470, 43998, 10674, 11423, 4, 46747, 21402, 7471, 45209, 27969, 10674, 1437, 47649, 711, 18537, 47649, 9264, 11423, 43998, 27969, 10470, 46747, 46, 711, 43998, 49171, 43998, 17772, 5782, 48444, 23133, 48444, 10470, 4, 46747, 711, 10470, 45209, 27969, 711, 46747, 27, 48, 46747, 711, 10470, 43998, 5782, 11423, 45209, 9264, 711, 43998, 15113, 10674, 116, 50118, 10431, 41761, 176, 10431, 35, 1437, 46873, 15389, 20024, 46873, 7487, 15722, 46873, 14292, 7471, 43998, 6248, 11936, 48280, 11936, 47672, 7487, 3726, 45209, 27969, 10674, 1437, 46873, 14292, 862, 48280, 20024, 46747, 7258, 13859, 48280, 11936, 1437, 46873, 14292, 862, 1437, 46873, 7487, 27, 43998, 15722, 11936, 43998, 11936, 48, 43998, 15113, 10674, 4, 50118, 10431, 41761, 134, 10431, 35, 1437, 46873, 18400, 18537, 45209, 21402, 6382, 

각 기능이 무엇을 의미하는지 좀 더 자세히 살펴보겠습니다.

• input_ids: 대화 상자에 매핑된 토큰 ID입니다. 각 토큰은 BART 모델에서 완벽하게 이해할 수 있는 단어 또는 하위 단어를 나타냅니다. 예를 들어, 숫자 5219는 BART 어휘에서 "hello"와 같은 단어에 대한 맵일 수 있습니다. 이 문맥에서는 각 단어마다 고유한 토큰이 있습니다.

• attention_mask: 이 마스크는 모델이 주의를 기울여야 하는 토큰과 무시해야 하는 토큰을 나타냅니다. 이는 문장의 길이를 균등화하기 위해 일부 토큰을 사용하는 경우 패딩의 맥락에서 자주 사용되지만 이러한 패딩 토큰의 대부분은 의미 있는 정보를 보유하지 않으므로 어텐션 마스크는 모델이 해당 토큰에 초점을 맞추지 않도록 합니다. 이 특정 샘플의 경우 모든 토큰은 '1'로 마스크되어 있습니다. 이는 모두 관련이 있으며 패딩에 사용되지 않음을 의미합니다.

• 레이블: 첫 번째 기능과 마찬가지로 요약의 단어와 하위 단어에서 얻은 토큰 ID입니다. 이는 모델이 출력으로 제공하기 위해 훈련될 토큰입니다.

이제 DataCollatorForSeq2Seq를 사용하여 데이터를 일괄 처리해야 합니다. 이러한 데이터 수집기는 패딩과 같은 일부 처리 기술을 자동으로 적용할 수도 있습니다. 모델을 미세 조정하는 작업에 중요하며 텍스트 요약을 위한 🤗 Transformers 문서에도 있습니다.

In [55]:
# # Instantiating Data Collator
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

다음으로 ROUGE 측정항목을 로드하고 모델을 평가하기 위한 새 함수를 정의하겠습니다.

Compute_metrics 함수는 설명서에서도 사용할 수 있습니다. 이 함수에서는 기본적으로 모델이 생성한 요약과 인간이 생성한 요약을 추출하여 디코딩합니다. 그런 다음 성능을 평가하기 위해 ROUGE를 사용하여 얼마나 유사한지 비교합니다.

In [56]:
# metric = load_metric('rouge') # Loading ROUGE Score

In [57]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred# Obtaining predictions and true labels
    
#     # Decoding predictions
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
#     # Obtaining the true labels tokens, while eliminating any possible masked token (i.e., label = -100)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Rouge expects a newline after each sentence
#     decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    
#     # Computing rouge score
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     result = {key: value.mid.fmeasure * 100 for key, value in result.items()} # Extracting some results

#     # Add mean-generated length
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)

#     return {k: round(v, 4) for k, v in result.items()}

이제 Seq2SeqTrainingArguments 클래스를 사용하여 미세 조정을 위한 일부 관련 설정을 지정합니다. 먼저 출력으로 사용할 디렉터리를 정의한 다음 평가 전략, 학습률 등을 정의하겠습니다.

이 클래스는 다양한 매개변수를 포함하여 매우 광범위할 수 있습니다. 문서를 숙지하기 위해 시간을 내어 문서를 살펴보는 것이 좋습니다.

In [58]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir = 'bart_samsum',
#     evaluation_strategy = "epoch",
#     save_strategy = 'epoch',
#     load_best_model_at_end = True,
#     metric_for_best_model = 'eval_loss',
#     seed = 42,
#     learning_rate=5e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=8,
#     weight_decay=0.01,
#     save_total_limit=2,
#     num_train_epochs=4,
#     predict_with_generate=True,
#     fp16=True,
#     report_to="none"
# )

마지막으로 Seq2SeqTrainer 클래스를 사용하면 PyTorch를 사용하여 모델을 미세 조정할 수 있습니다. 이 수업에서는 기본적으로 모델, 훈련 인수, 훈련 및 평가에 사용되는 데이터세트, 토크나이저, data_collator 및 측정항목을 정의합니다.

In [59]:
# # Defining Trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_val, #tokenized_test,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [60]:
# trainer.train() # Training model

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,0.653986,68.102200,28.239300,58.418200,59.711400,61.384800
1,0.743500,0.607286,63.736800,24.847800,54.752100,55.959900,61.452900
2,0.552100,0.568845,65.175900,29.178400,57.925600,59.265000,61.701400
3,0.489300,0.554441,64.086500,27.394800,56.673500,58.351600,61.396800


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1548, training_loss=0.5908482955715761, metrics={'train_runtime': 3192.4174, 'train_samples_per_second': 15.543, 'train_steps_per_second': 0.485, 'total_flos': 9.789895886566195e+16, 'train_loss': 0.5908482955715761, 'epoch': 3.99})

우리는 4번의 에포크를 거쳐 마침내 미세 조정을 마쳤습니다. 훈련 인수에 load_best_model_at_end = True가 있었기 때문에 Trainer는 자동으로 최고의 성능을 가진 모델을 저장합니다. 이 경우 검증 손실이 가장 낮은 모델입니다.

두 번째 에포크는 검증 손실이 가장 낮은 1.443861이었습니다. 루즈1, 루즈2 점수도 가장 높았고, 루겔섬 점수도 가장 높았다.

나는 이전에 Rougelsum 점수를 제시한 적이 없습니다. rouge-score 라이브러리의 문서에 따르면 이는 RougeL 점수와 유사하다고 결론을 내릴 수 있지만 전체 요약이 아닌 문장별 수준에서 콘텐츠 범위를 측정합니다.

Gen Len 열은 모델 생성 요약의 평균 길이를 제공합니다. 우리는 짧지만 유익한 텍스트를 원한다는 점을 기억하는 것이 중요합니다. 이 경우에도 두 번째 에포크는 평균적으로 가장 짧은 요약을 산출했습니다.

# Evaluating and Saving Model

모델을 훈련하고 테스트한 후 검증 데이터 세트에서 성능을 평가할 수 있습니다. 이를 위해 평가 방법을 사용할 수 있습니다.